In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import joblib
import imodelsx.process_results
import sys
from sklearn.metrics import accuracy_score

# def load_csvs(path):

#     X_train = pd.read_csv(f'{path}/X_trainval.csv', index_col=0)
#     X_train_hat = pd.read_csv(f'{path}/X_trainval_hat.csv', index_col=0)
#     X_test = pd.read_csv(f'{path}/X_test.csv', index_col=0)
#     X_test_hat = pd.read_csv(f'{path}/X_test_hat.csv', index_col=0)
#     y_train = pd.read_csv(f'{path}/y_trainval.csv', index_col=0)
#     y_train_hat = pd.read_csv(f'{path}/y_trainval_hat.csv', index_col=0)
#     y_test = pd.read_csv(f'{path}/y_test.csv', index_col=0)
#     y_test_hat = pd.read_csv(f'{path}/y_test_hat.csv', index_col=0)

#     return X_train, X_train_hat, X_test, X_test_hat, y_train, y_train_hat, y_test, y_test_hat

sys.path.append('../experiments/')
results = []

results_dir = f'../results/cub_distillation_edit'
experiment_filename = '../experiments/cub_distillation_edit.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

2025-01-03 11:38:43.756117: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-03 11:38:44.823020: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mattyshen/anaconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 503.52it/s]


experiment varied these params: []


ValueError: No group keys passed!

In [2]:
r

,task_type,model_name,X_type,thresh,Y_type,save_dir,max_rules,max_trees,max_depth,pre_interaction,...,%_correct_seed3_overlap_trainval,%_correct_seed3_overlap_test,edited_cbm_true_seed3_accuracy_trainval,edited_cbm_true_seed3_accuracy_test,edited_distiller_true_seed3_accuracy_trainval,edited_distiller_true_seed3_accuracy_test,edited_distiller_cbm_seed3_accuracy_trainval,edited_distiller_cbm_seed3_accuracy_test,edited_%_correct_seed3_overlap_trainval,edited_%_correct_seed3_overlap_test
0,regression,FIGSRegressor,binary,0.25,logits,/home/mattyshen/DistillationEdit/results/cub_d...,60,30,3,l0l2,...,0.983289,0.935278,0.983623,0.783569,0.976103,0.762858,0.983456,0.891785,0.98379,0.935105


In [ ]:
#TODO: clean below when results are logged

In [2]:
r['cbm_mean_accuracy_val'] = np.mean([r[f'cbm_seed{seed}_accuracy_val'] for seed in range(1, 4)], axis = 0)
r['cbm_std_accuracy_val'] = np.std([r[f'cbm_seed{seed}_accuracy_val'] for seed in range(1, 4)], axis = 0)
r['cbm_mean_accuracy_train'] = np.mean([r[f'cbm_seed{seed}_accuracy_train'] for seed in range(1, 4)], axis = 0)
r['cbm_std_accuracy_train'] = np.std([r[f'cbm_seed{seed}_accuracy_train'] for seed in range(1, 4)], axis = 0)

r['true_mean_accuracy_val'] = np.mean([r[f'true_seed{seed}_accuracy_val'] for seed in range(1, 4)], axis = 0)
r['true_std_accuracy_val'] = np.std([r[f'true_seed{seed}_accuracy_val'] for seed in range(1, 4)], axis = 0)
r['true_mean_accuracy_train'] = np.mean([r[f'true_seed{seed}_accuracy_train'] for seed in range(1, 4)], axis = 0)
r['true_std_accuracy_train'] = np.std([r[f'true_seed{seed}_accuracy_train'] for seed in range(1, 4)], axis = 0)

In [3]:
r['thresh'] = r['thresh'].fillna(0) + r['half'].fillna(0)*0.5

In [4]:
r = r.drop(columns  = ['save_dir', 'save_dir_unique', 'use_cache', 'half']
      +[f'true_seed{seed}_accuracy_val' for seed in range(1, 4)]+
       [f'true_seed{seed}_accuracy_train' for seed in range(1, 4)]+
       [f'cbm_seed{seed}_accuracy_val' for seed in range(1, 4)]+
       [f'cbm_seed{seed}_accuracy_train' for seed in range(1, 4)])

In [5]:
r[r['model_name'] == 'elastic_net'].drop(columns=['task_type', 'max_depth', 'pre_max_features', 'max_rules', 'reg_depth','shrink_depth', 'reg_shrink'])

,model_name,X_type,Y_type,max_trees,pre_interaction,post_interaction,post_max_features,thresh,mo,cbm_mean_accuracy_val,cbm_std_accuracy_val,cbm_mean_accuracy_train,cbm_std_accuracy_train,true_mean_accuracy_val,true_std_accuracy_val,true_mean_accuracy_train,true_std_accuracy_train
49,elastic_net,binary,probs,30,l0l2,l0l2,30.0,0.0,False,0.797607,0.004439,0.984347,0.002682,0.704925,0.003870,0.972315,0.000284
69,elastic_net,binary,logits,30,l0l2,l0l2,30.0,0.0,False,0.858647,0.006432,0.992647,0.002367,0.742147,0.003612,0.988581,0.001570
117,elastic_net,probs,logits,30,l0l2,l0l2,30.0,0.0,False,0.995858,0.000508,0.999499,0.000361,0.776953,0.002638,0.984737,0.001817
166,elastic_net,probs,probs,30,l0l2,l0l2,30.0,0.0,False,0.936083,0.005144,0.988692,0.003415,0.772408,0.005568,0.974933,0.001774


In [6]:
r.loc[(r['model_name'] == 'ft_distill') & (r['task_type'] == 'classification'), 'model_name'] = 'FTDistillClassifier'
r.loc[(r['model_name'] == 'ft_distill') & (r['task_type'] == 'regression'), 'model_name'] = 'FTDistillHydraRegressor'
r.loc[(r['model_name'] == 'FTDistillHydraRegressor') & (r['max_depth'] == 1), 'model_name'] = 'FTDistillHydraRegressor (No Interactions)'
r.loc[(r['model_name'] == 'decision_tree') & (r['task_type'] == 'regression'), 'model_name'] = 'DTRegressor'
r.loc[(r['model_name'] == 'decision_tree') & (r['task_type'] == 'classification'), 'model_name'] = 'DTClassifier'
r.loc[(r['model_name'] == 'random_forest') & (r['task_type'] == 'regression'), 'model_name'] = 'RFRegressor'
r.loc[(r['model_name'] == 'random_forest') & (r['task_type'] == 'classification'), 'model_name'] = 'RFClassifier'

In [7]:
r = r[r.model_name != 'elastic_net']

In [8]:
plotting = r.groupby(['model_name', 'task_type']).apply(lambda sdf: sdf.sort_values(by='cbm_mean_accuracy_val', ascending=False).iloc[0,:]).drop(columns=['model_name', 'task_type']).reset_index()

In [9]:
plotting.reset_index().drop(columns=['reg_shrink', 'max_rules', 'max_trees', 'pre_interaction'])

,index,model_name,task_type,X_type,Y_type,max_depth,pre_max_features,post_interaction,post_max_features,thresh,...,shrink_depth,mo,cbm_mean_accuracy_val,cbm_std_accuracy_val,cbm_mean_accuracy_train,cbm_std_accuracy_train,true_mean_accuracy_val,true_std_accuracy_val,true_mean_accuracy_train,true_std_accuracy_train
0,0,DTClassifier,classification,binary,classes,8,1.0,l0l2,30.0,0.0,...,0.0,False,0.194684,0.000924,0.265319,0.007489,0.167702,0.001681,0.252117,0.008919
1,1,DTRegressor,regression,probs,logits,8,1.0,l0l2,30.0,0.0,...,0.0,False,0.493326,0.028981,0.679757,0.021957,0.439938,0.017237,0.671234,0.016667
2,2,FIGSClassifier,classification,probs,classes,3,1.0,l0l2,30.0,0.0,...,0.0,False,0.582269,0.170401,0.732620,0.188379,0.514268,0.143457,0.719753,0.187564
3,3,FIGSHydraRegressor,regression,binary,logits,4,1.0,l0l2,30.0,0.0,...,0.0,False,0.713036,0.013265,0.923240,0.008356,0.649005,0.003624,0.923908,0.006993
4,4,FIGSRegressor,regression,binary,logits,4,1.0,l0l2,30.0,0.5,...,0.0,False,0.927914,0.002185,0.994374,0.002633,0.765102,0.002912,0.981061,0.004886
5,5,FTDistillClassifier,classification,probs,classes,1,112.0,l0l2,112.0,0.0,...,0.0,False,0.954033,0.004138,0.999276,0.000343,0.774709,0.002053,0.984626,0.001910
6,6,FTDistillHydraRegressor,regression,probs,probs,2,1.0,l0l2,7.0,0.0,...,0.0,True,0.870958,0.014137,0.981228,0.005957,0.743470,0.005416,0.972037,0.001853
7,7,FTDistillHydraRegressor (No Interactions),regression,probs,probs,1,1.0,l0l2,7.0,0.0,...,0.0,True,0.783684,0.015644,0.855002,0.017749,0.675526,0.010141,0.845978,0.013120
8,8,RFClassifier,classification,probs,classes,8,1.0,l0l2,30.0,0.0,...,0.0,False,0.903118,0.013839,0.996156,0.002363,0.759004,0.006117,0.981339,0.000776
9,9,RFRegressor,regression,probs,probs,4,1.0,l0l2,30.0,0.0,...,0.0,False,0.562939,0.010285,0.632242,0.004884,0.499425,0.004951,0.625334,0.002131


In [10]:
r[r.model_name == 'FTDistillHydraRegressor'].sort_values('cbm_mean_accuracy_val', ascending=False)

,task_type,model_name,X_type,Y_type,max_rules,max_trees,max_depth,pre_interaction,pre_max_features,post_interaction,...,reg_shrink,mo,cbm_mean_accuracy_val,cbm_std_accuracy_val,cbm_mean_accuracy_train,cbm_std_accuracy_train,true_mean_accuracy_val,true_std_accuracy_val,true_mean_accuracy_train,true_std_accuracy_train
68,regression,FTDistillHydraRegressor,probs,probs,60,30,2,None,1.0,l0l2,...,0.0,True,0.870958,0.014137,0.981228,0.005957,0.743470,0.005416,0.972037,0.001853
56,regression,FTDistillHydraRegressor,binary,probs,60,30,2,None,1.0,l0l2,...,0.0,True,0.858129,0.002188,0.980559,0.007118,0.734265,0.004109,0.976771,0.002482
170,regression,FTDistillHydraRegressor,probs,probs,60,30,2,None,1.0,l0l2,...,0.0,True,0.857726,0.012074,0.977941,0.003320,0.736912,0.004769,0.966355,0.003052
266,regression,FTDistillHydraRegressor,binary,probs,60,30,2,None,1.0,l0l2,...,0.0,True,0.853584,0.003354,0.981729,0.005494,0.739673,0.001279,0.976660,0.002597
284,regression,FTDistillHydraRegressor,binary,probs,60,30,2,None,1.0,l0l2,...,0.0,True,0.852951,0.004474,0.977997,0.010040,0.726556,0.003324,0.968471,0.009077
203,regression,FTDistillHydraRegressor,binary,probs,60,30,2,None,1.0,l0l2,...,0.0,True,0.851168,0.002690,0.976883,0.006315,0.735473,0.000215,0.967525,0.006124
47,regression,FTDistillHydraRegressor,binary,logits,60,30,2,None,1.0,l0l2,...,0.0,True,0.830687,0.015461,0.927362,0.016961,0.720746,0.012448,0.927418,0.017584
44,regression,FTDistillHydraRegressor,binary,logits,60,30,2,None,1.0,l0l2,...,0.0,True,0.805086,0.023392,0.911096,0.031481,0.705960,0.018109,0.913603,0.026958
248,regression,FTDistillHydraRegressor,probs,logits,60,30,2,None,1.0,l0l2,...,0.0,True,0.751467,0.006668,0.922906,0.017556,0.664423,0.006794,0.917502,0.012928
115,regression,FTDistillHydraRegressor,binary,logits,60,30,2,None,1.0,l0l2,...,0.0,True,0.704695,0.023933,0.780916,0.027015,0.619722,0.015847,0.783701,0.028518


In [11]:
cbm_plot = plotting[['model_name', 'cbm_mean_accuracy_train', 'cbm_mean_accuracy_val']].set_index('model_name').round(2).rename(columns = {'cbm_mean_accuracy_val':'test accuracy', 'cbm_mean_accuracy_train':'train accuracy'})

In [12]:
true_plot = plotting[['model_name', 'true_mean_accuracy_train', 'true_mean_accuracy_val']].set_index('model_name').round(2).rename(columns = {'true_mean_accuracy_val':'test accuracy', 'true_mean_accuracy_train':'train accuracy'})

In [13]:
#true = plotting[['model_name', 'true_mean_accuracy_train', 'true_mean_accuracy_val']].set_index('model_name').round(2).rename(columns = {'true_mean_accuracy_val':'test accuracy', 'true_mean_accuracy_train':'train accuracy'})

In [14]:
cbm_scores = np.zeros((3, 2))

for seed in range(1, 4):
    X_train, X_train_hat, X_test, X_test_hat, y_train, y_train_hat, y_test, y_test_hat = load_csvs(f'/home/mattyshen/interpretableDistillation/interpretDistill/data/cbm_datasets/seed0_Joint0.01SigmoidModel__Seed{seed}')
    cbm_scores[seed-1, 0] = accuracy_score(y_train_hat.idxmax(axis=1).astype(int), y_train)
    cbm_scores[seed-1, 1] = accuracy_score(y_test_hat.idxmax(axis=1).astype(int), y_test)

cbm_scores.mean(axis = 0)

array([0.98468137, 0.77712576])

In [15]:
true_plot.loc['CBM'] = cbm_scores.mean(axis = 0)

In [16]:
true_plot = true_plot.sort_values('test accuracy', ascending=False)
cbm_plot = cbm_plot.sort_values('test accuracy', ascending=False)

In [17]:
cbm_plot.round(2)

,train accuracy,test accuracy
model_name,,
FTDistillClassifier,1.00,0.95
FIGSRegressor,0.99,0.93
xgboost,0.99,0.93
RFClassifier,1.00,0.90
FTDistillHydraRegressor,0.98,0.87
FTDistillHydraRegressor (No Interactions),0.86,0.78
FIGSHydraRegressor,0.92,0.71
FIGSClassifier,0.73,0.58
RFRegressor,0.63,0.56


In [24]:
r[r.model_name=='xgboost'].drop(columns=['max_rules', 'max_trees', 'pre_interaction', 'pre_max_features', 'post_interaction', 'post_max_features', 'thresh', 'reg_depth','shrink_depth','reg_shrink','mo'])

,task_type,model_name,X_type,Y_type,max_depth,cbm_mean_accuracy_val,cbm_std_accuracy_val,cbm_mean_accuracy_train,cbm_std_accuracy_train,true_mean_accuracy_val,true_std_accuracy_val,true_mean_accuracy_train,true_std_accuracy_train
39,regression,xgboost,binary,probs,8,0.766310,0.004516,0.997270,0.000877,0.681279,0.004935,0.984514,0.002442
51,regression,xgboost,binary,logits,3,0.926188,0.005709,0.993928,0.003256,0.762053,0.003092,0.981061,0.002325
70,regression,xgboost,probs,logits,3,0.811299,0.011422,0.989862,0.001268,0.710850,0.009915,0.983456,0.001438
73,regression,xgboost,binary,logits,3,0.912611,0.005422,0.994931,0.002480,0.762916,0.004070,0.981228,0.004058
97,regression,xgboost,binary,probs,3,0.900587,0.007519,0.995766,0.001111,0.750834,0.004314,0.982063,0.001616
132,regression,xgboost,binary,probs,8,0.893223,0.006315,0.997828,0.001117,0.755897,0.005330,0.983957,0.002399
150,regression,xgboost,binary,probs,8,0.902658,0.006014,0.997716,0.000834,0.752733,0.004221,0.985684,0.001853
180,regression,xgboost,probs,probs,8,0.598550,0.018708,0.999610,0.000079,0.546888,0.017172,0.984514,0.001969
187,regression,xgboost,probs,probs,3,0.607237,0.014322,0.998830,0.000546,0.553504,0.011315,0.984793,0.001660
219,regression,xgboost,binary,probs,3,0.756472,0.005771,0.994652,0.000955,0.671039,0.008428,0.983512,0.003752


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_accuracy(df, title, mean_val_col, std_val_col, mean_train_col, std_train_col):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Bar positions
    indices = np.arange(len(df['model_name']))

    # Bar widths
    bar_width = 0.35

    # Plot bars for validation accuracy
    ax.bar(indices, df[mean_val_col], bar_width, yerr=df[std_val_col], label='Test Accuracy', capsize=5)

    # Plot bars for training accuracy
    ax.bar(indices + bar_width, df[mean_train_col], bar_width, yerr=df[std_train_col], label='Training/Validation Accuracy', capsize=5)

    # Add some text for labels, title, and axes ticks
    ax.set_xticklabels(indices, rotation=45, fontsize=7)
    ax.set_xlabel('Model Name')
    ax.set_ylabel('Accuracy')
    ax.set_title(title)
    ax.set_xticks(indices + bar_width / 2)
    ax.set_xticklabels(df['model_name'])
    ax.legend()

    plt.tight_layout()
    plt.savefig('cbm_distillation_accuracy.png')
    plt.show()

# Plot for CBM accuracies
plot_accuracy(plotting.sort_values('cbm_mean_accuracy_val', ascending=False), 'CBM Distillation Accuracy', 'cbm_mean_accuracy_val', 'cbm_std_accuracy_val', 'cbm_mean_accuracy_train', 'cbm_std_accuracy_train')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_accuracy_with_hlines(df, title, mean_val_col, std_val_col, mean_train_col, std_train_col, cbm_scores):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Bar positions
    indices = np.arange(len(df['model_name']))

    # Bar widths
    bar_width = 0.35

    # Plot bars for validation accuracy
    ax.bar(indices, df[mean_val_col], bar_width, yerr=df[std_val_col], label='Test Accuracy', capsize=5)

    # Plot bars for training accuracy
    ax.bar(indices + bar_width, df[mean_train_col], bar_width, yerr=df[std_train_col], label='Training/Validation Accuracy', capsize=5)

    # Add horizontal lines for train and test scores with std from cbm_scores
    train_mean = cbm_scores[:, 0].mean()
    train_std = cbm_scores[:, 0].std()
    test_mean = cbm_scores[:, 1].mean()
    test_std = cbm_scores[:, 1].std()

    ax.axhline(y=train_mean, color='orange', linestyle='--', label=f'CBM Train/Val Accuracy: {train_mean:.2f}')
    ax.fill_between([indices[0] - bar_width, indices[-1] + bar_width * 2], 
                    train_mean - train_std, train_mean + train_std, color='orange', alpha=0.2)

    # Horizontal line for test scores with std
    ax.axhline(y=test_mean, color='blue', linestyle='--', label=f'CBM Test Accuracy: {test_mean:.2f}')
    ax.fill_between([indices[0] - bar_width, indices[-1] + bar_width * 2], 
                    test_mean - test_std, test_mean + test_std, color='blue', alpha=0.2)

    # Add labels, title, and legend
    ax.set_xticklabels(indices, rotation=45, fontsize=7)
    ax.set_xlabel('Model Name')
    ax.set_ylabel('Accuracy')
    ax.set_title(title)
    ax.set_xticks(indices + bar_width / 2)
    ax.set_xticklabels(df['model_name'])
    ax.legend()

    plt.tight_layout()
    plt.savefig('cbm_true_accuracy.png')
    plt.show()

# Plot for True accuracies with horizontal lines
plot_accuracy_with_hlines(plotting.sort_values('cbm_mean_accuracy_val', ascending=False), 'True Accuracy with CBM Accuracy', 
                          'true_mean_accuracy_val', 'true_std_accuracy_val', 
                          'true_mean_accuracy_train', 'true_std_accuracy_train', cbm_scores)


In [ ]:
r[r["model_name"]=="FIGSRegressor"].sort_values('cbm_mean_accuracy_val', ascending=False).drop(columns=['task_type', 'model_name','save_dir', 
                                                                                                     'save_dir_unique', 'use_cache']+
                                                                                           [f'true_seed{seed}_accuracy_val' for seed in range(1, 4)]+
                                                                                           [f'true_seed{seed}_accuracy_train' for seed in range(1, 4)]+
                                                                                           [f'cbm_seed{seed}_accuracy_val' for seed in range(1, 4)]+
                                                                                           [f'cbm_seed{seed}_accuracy_train' for seed in range(1, 4)])

In [ ]:
r[r["model_name"]=="ft_distill"].sort_values('cbm_mean_accuracy_val', ascending=False).drop(columns=['task_type', 'model_name', 
                                                                                                     'X_type', 'Y_type', 'save_dir', 
                                                                                                     'save_dir_unique', 'use_cache',
                                                                                                    'max_rules', 'max_trees']+
                                                                                           [f'true_seed{seed}_accuracy_val' for seed in range(1, 4)]+
                                                                                           [f'true_seed{seed}_accuracy_train' for seed in range(1, 4)]+
                                                                                           [f'cbm_seed{seed}_accuracy_val' for seed in range(1, 4)]+
                                                                                           [f'cbm_seed{seed}_accuracy_train' for seed in range(1, 4)]) #[['pre_interaction', 'pre_max_features', 'post_interaction', 'post_max_features']]

In [ ]:
r['figs_training_time'].mean(), r['ftd_training_time'].mean()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


def plot_histogram_for_dataset(df, dataset_name, train=False):
    # Set the aesthetics for the plots
    sns.set_context("talk", font_scale=5)
    sns.set(style="whitegrid")
    
    train_str = 'train' if train else 'val'

    # Create a catplot for model_name and distiller_name
    sns.set(font_scale=1.25)
    g = sns.catplot(
        data=df,
        x='dataset_name',
        y=f'{train_str}_r2',
        hue='model_name',
        kind='bar',
        height=6,
        aspect=2,
    )
    
    # Set the title and labels
    if train:
        g.fig.suptitle(f'Best Train R2 Scores for FIGS, FT Distill, Ridge', fontsize=25)
        g.set_axis_labels("Model Name", "Train R2 Score", fontsize=25)
    else:
        g.fig.suptitle(f'Best Val R2 Scores for FIGS, FT Distill, Ridge', fontsize=25)
        g.set_axis_labels("Model Name", "Val R2 Score", fontsize=25)
    #g._legend.set_title('Distiller Name')
    sns.move_legend(g, bbox_to_anchor=(1,0.5), loc="center left", markerscale=5, title="model")
    
    # Adjust the layout
    plt.yticks(ticks=np.arange(0, 1.09, 0.1))
    plt.xticks(rotation=45)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    g.savefig(f'plots/{dataset_name}_{train_str}_r2.png', bbox_inches='tight')
#hue=df[['distiller_name', 'binary_mapper_name']].apply(tuple, axis =1)

In [ ]:
r['max_rules'].value_counts()

In [ ]:
np.argmin(r['ftd_r2_score_val']/r['figs_r2_score_val'])

In [ ]:
r.iloc[73, :]

In [ ]:
plt.hist((r[r['max_rules'] == 20]['ftd_r2_score_val']/r[r['max_rules'] == 20]['figs_r2_score_val'])*100 - 100)

In [ ]:
plt.hist((r[r['max_rules'] == 30]['ftd_r2_score_val']/r[r['max_rules'] == 30]['figs_r2_score_val'])*100-100)

In [ ]:
plt.hist((r['ftd_r2_score_val']/r['figs_r2_score_val'])*100-100, bins=50)

In [ ]:
metrics = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='ftd_r2_score_val', ascending=False).iloc[0,:])[['figs_max_interaction_size', 'ftd_max_interaction_size', 'num_common_interactions', 'figs_training_time' , 'ftd_training_time']]

In [ ]:
metrics = metrics.reset_index()

In [ ]:
df_melted = pd.melt(metrics, id_vars='dataset_name', value_vars=['figs_max_interaction_size', 'ftd_max_interaction_size'],
                    var_name='model_name', value_name='max_interaction_size')
fig = plt.figure(figsize=(15,10))
# Create a categorical plot
g = sns.catplot(
    data=df_melted, 
    x='dataset_name', 
    y='max_interaction_size', 
    hue='model_name', 
    kind='bar', 
    height=5, 
    aspect=2
)

# Set plot title and labels
plt.title('Max Interaction Size by Dataset for Best Val R2 Model')
plt.xlabel('Dataset Name')
plt.ylabel('Max Interaction Size')

# Show the plot
plt.show()
g.savefig(f'plots/interaction_size.png', bbox_inches='tight')

In [ ]:
int_df

In [ ]:
int_df = r.groupby(['dataset_name'])[['figs_max_interaction_size', 'ftd_max_interaction_size']].mean().reset_index().rename(columns={'figs_max_interaction_size':'figs', 'ftd_max_interaction_size':'ftd'})
df_melted = pd.melt(int_df, id_vars='dataset_name', value_vars=['figs', 'ftd'],
                    var_name='model_name', value_name='max_interaction_size')

# Create a categorical plot
g = sns.catplot(
    data=df_melted, 
    x='dataset_name', 
    y='max_interaction_size', 
    hue='model_name', 
    kind='bar', 
    height=5, 
    aspect=2
)

# Set plot title and labels
plt.title('Mean Max Interaction Size by Dataset')
plt.xlabel('Dataset Name')
plt.ylabel('Mean Max Interaction Size')

# Show the plot
plt.show()
g.savefig(f'plots/interaction_size.png', bbox_inches='tight')

In [ ]:
ftd = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='ftd_r2_score_val', ascending=False).iloc[0,:])[['ftd_r2_score_train', 'ftd_r2_score_val']]
figs = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='figs_r2_score_val', ascending=False).iloc[0,:])[['figs_r2_score_train', 'figs_r2_score_val']]
ridge_decoup = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='ridge_figs_decoup_r2_score_val', ascending=False).iloc[0,:])[['ridge_figs_decoup_r2_score_train', 'ridge_figs_decoup_r2_score_val']]
ridge_inter = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='ridge_figs_inter_r2_score_val', ascending=False).iloc[0,:])[['ridge_figs_inter_r2_score_train', 'ridge_figs_inter_r2_score_val']]
xgb = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='xgboost_r2_score_val', ascending=False).iloc[0,:])[['xgboost_r2_score_train', 'xgboost_r2_score_val']]

In [ ]:
ftd['model_name'] = 'ft_distill'
figs['model_name'] = 'figs'
ridge_decoup['model_name'] = 'ridge_decoup'
ridge_inter['model_name'] = 'ridge_inter'
xgb['model_name'] = 'xgboost'
ftd = ftd.rename(columns = {'ftd_r2_score_train':'train_r2', 'ftd_r2_score_val':'val_r2'})
figs= figs.rename(columns = {'figs_r2_score_train':'train_r2', 'figs_r2_score_val':'val_r2'})
ridge_decoup= ridge_decoup.rename(columns = {'ridge_figs_decoup_r2_score_train':'train_r2', 'ridge_figs_decoup_r2_score_val':'val_r2'})
ridge_inter= ridge_inter.rename(columns = {'ridge_figs_inter_r2_score_train':'train_r2', 'ridge_figs_inter_r2_score_val':'val_r2'})
xgb= xgb.rename(columns = {'xgboost_r2_score_train':'train_r2', 'xgboost_r2_score_val':'val_r2'})

In [ ]:
models = pd.concat([ftd, figs, ridge_decoup, ridge_inter, xgb], axis = 0)
models.reset_index(inplace=True)

In [ ]:
def show_p_improve(base, comp, round_degree = 2):
    core = np.round((comp.drop(columns = {'model_name'}).values/base.drop(columns = {'model_name'}).values), round_degree)*100-100
    return pd.DataFrame(core, columns = comp.columns[:-1]).set_index([comp.index])

In [ ]:
show_p_improve(figs, ftd)

In [ ]:
show_p_improve(figs, ridge_decoup)

In [ ]:
show_p_improve(figs, ridge_inter)

In [ ]:
plot_histogram_for_dataset(models, 'figs_sim', train=False)

In [ ]:
r['num_common_interactions'].value_counts()

In [ ]:
int_df = r.groupby(['dataset_name'])[['figs_max_interaction_size', 'ftd_max_interaction_size']].mean().reset_index().rename(columns={'figs_max_interaction_size':'figs', 'ftd_max_interaction_size':'ftd'})
df_melted = pd.melt(int_df, id_vars='dataset_name', value_vars=['figs', 'ftd'],
                    var_name='model_name', value_name='max_interaction_size')

# Create a categorical plot
g = sns.catplot(
    data=df_melted, 
    x='dataset_name', 
    y='max_interaction_size', 
    hue='model_name', 
    kind='bar', 
    height=5, 
    aspect=2
)

# Set plot title and labels
plt.title('Mean Max Interaction Size by Dataset')
plt.xlabel('Dataset Name')
plt.ylabel('Mean Max Interaction Size')

# Show the plot
plt.show()
g.savefig(f'plots/interaction_size.png', bbox_inches='tight')

In [ ]:
r['num_common_interactions'].value_counts()

In [ ]:
r['ftd_max_interaction_size'].value_counts()

In [ ]:
ftd = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='ftd_r2_score_val', ascending=False).iloc[0,:])[['ftd_r2_score_train', 'ftd_r2_score_val']] #[['ftd_r2_score_train', 'ftd_r2_score_val', 'figs_r2_score_train', 'figs_r2_score_val']].max()

In [ ]:
figs = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='figs_r2_score_val', ascending=False).iloc[0,:])[['figs_r2_score_train', 'figs_r2_score_val']] #[['ftd_r2_score_train', 'ftd_r2_score_val', 'figs_r2_score_train', 'figs_r2_score_val']].max()

In [ ]:
ridge = r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='ridge_r2_score_val', ascending=False).iloc[0,:])[['ridge_r2_score_train', 'ridge_r2_score_val']] #[['ftd_r2_score_train', 'ftd_r2_score_val', 'figs_r2_score_train', 'figs_r2_score_val']].max()

In [ ]:
ftd['model_name'] = 'ft_distill'
figs['model_name'] = 'figs'
ridge['model_name'] = 'ridge'
ftd = ftd.rename(columns = {'ftd_r2_score_train':'train_r2', 'ftd_r2_score_val':'val_r2'})
figs= figs.rename(columns = {'figs_r2_score_train':'train_r2', 'figs_r2_score_val':'val_r2'})
ridge= ridge.rename(columns = {'ridge_r2_score_train':'train_r2', 'ridge_r2_score_val':'val_r2'})

In [ ]:
models = pd.concat([ftd, figs, ridge], axis = 0)
models.reset_index(inplace=True)

In [ ]:
plot_histogram_for_dataset(models, 'figs_sim', train=False)

In [ ]:
r.groupby(['dataset_name']).apply(lambda sdf: sdf.sort_values(by='ridge_r2_score_val', ascending=False).iloc[0,:])[['dataset_name', 'binary_mapper_frac', 'ridge_r2_score_train', 'ridge_r2_score_val', 'max_features']] #[['ftd_r2_score_train', 'ftd_r2_score_val', 'figs_r2_score_train', 'figs_r2_score_val']].max()

In [ ]:
df = r.groupby(['model_name', 'distiller_name', 'binary_mapper_name'])['val_r2'].mean().reset_index() #.apply(lambda sdf: sdf.sort_values(by='val_r2', ascending=False).iloc[0,:])
df.loc[:, 'distiller+binary_mapper'] = df['distiller_name'] + ' + '+ df['binary_mapper_name'] #(df['distiller_name'] == 'None').map(lambda x: 'original_model' if x else '') + (df['distiller_name'] + " + " + df['binary_mapper_name'])*(df['distiller_name'] != 'None').to_numpy()
plot_histogram_for_dataset(df, 'all datasets')

In [ ]:
r['n_epochs'].value_counts()

In [ ]:
r[r['model_name'] == 'ft_distill']

In [ ]:
datasets = r['dataset_name'].unique()
datasets

In [ ]:
r.groupby(['dataset_name', 'binary_mapper_name'])[['teacher_r2_score_train_true', 'distiller_r2_score_train_true']].mean()

In [ ]:
r[(r['distiller_name']=='ft_distill') & (r['max_depth']==5) & (r['max_features']==0.75)] #['max_features'].value_counts()

In [ ]:
#RF params: max_depth, max_features
random_forest= r[r['model_name'] == 'random_forest'].drop(columns=['subsample_frac', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'n_epochs', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions'])
random_forest_g = random_forest.groupby(['dataset_name','model_name', 'max_depth', 'max_features', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()

In [ ]:
random_forest.groupby(['max_depth', 'max_features', 'seed']).count()

In [ ]:
# random_forest_best_hyp = {}
# for d in datasets:
#     queried = random_forest_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name', 'max_depth', 'max_features'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     random_forest_best_hyp[d] = d_best_hyp
# random_forest_best_hyp

In [ ]:
#RF+ params: max_depth, max_features
rf_plus = r[r['model_name'] == 'rf_plus'].drop(columns=['subsample_frac', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'n_epochs', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions'])
rf_plus_g = rf_plus.groupby(['dataset_name','model_name', 'max_depth', 'max_features', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()
rf_plus_g

In [ ]:
rf_plus.groupby(['max_depth', 'max_features', 'seed']).count()

In [ ]:
# rf_plus_best_hyp = {}
# for d in datasets:
#     queried = rf_plus_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name', 'max_depth', 'max_features'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     rf_plus_best_hyp[d] = d_best_hyp
# rf_plus_best_hyp

In [ ]:
#FIGS params: max_rules, max_trees, max_features
figs = r[r['model_name'] == 'figs'].drop(columns=['subsample_frac', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'n_epochs', 'max_depth','pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions'])
figs_g = figs.groupby(['dataset_name','model_name', 'max_rules','max_trees', 'max_features', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean().round(2)
figs_g

In [ ]:
figs.groupby(['max_rules', 'max_trees', 'max_features', 'seed']).count()

In [ ]:
#RF+ params: max_depth, max_features
xgboost = r[r['model_name'] == 'xgboost'].drop(columns=['subsample_frac', 'save_dir','use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'n_epochs', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions', 'max_features'])
xgboost_g = xgboost.groupby(['dataset_name','model_name', 'max_depth', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean().round(2)
xgboost_g

In [ ]:
xgboost.groupby(['max_depth', 'seed']).count()

In [ ]:
# xgboost_best_hyp = {}
# for d in datasets:
#     queried = xgboost_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name', 'max_depth'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     xgboost_best_hyp[d] = d_best_hyp
# xgboost_best_hyp

In [ ]:
#ResNet params: n_epochs
resnet = r[r['model_name'] == 'resnet'].drop(columns=['subsample_frac', 'save_dir','use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions', 'max_features', 'max_depth'])
resnet_g = resnet.groupby(['dataset_name','model_name', 'n_epochs', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()
resnet_g

In [ ]:
resnet.groupby(['n_epochs', 'seed']).count()

In [ ]:
# resnet_best_hyp = {}
# for d in resnet['dataset_name'].unique():
#     queried = resnet_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name','n_epochs'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     resnet_best_hyp[d] = d_best_hyp
# resnet_best_hyp

In [ ]:
#FT Transformer params: n_epochs
ft_transformer = r[r['model_name'] == 'ft_transformer'].drop(columns=['subsample_frac', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions', 'max_features', 'max_depth'])
ft_transformer_g = ft_transformer.groupby(['dataset_name','model_name', 'n_epochs', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()
ft_transformer_g

In [ ]:
ft_transformer.groupby(['n_epochs', 'seed']).count()

In [ ]:
# ft_transformer_best_hyp = {}
# for d in ft_transformer['dataset_name'].unique():
#     queried = ft_transformer_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name','n_epochs'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     ft_transformer_best_hyp[d] = d_best_hyp
# ft_transformer_best_hyp

In [ ]:
best_hyp = {}
for m in ['random_forest', 'rf_plus', 'figs', 'xgboost', 'resnet', 'ft_transformer']:
    best_hyp[m] = eval(f'{m}_best_hyp')

In [ ]:
import json 

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

with open('/home/mattyshen/interpretableDistillation/scripts/best_hyperparams/original_hyperparams.json', "w") as outfile: 
    json.dump(best_hyp, outfile, cls =NpEncoder)